In [17]:
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.indicators.gd import GD
from pymoo.indicators.igd import IGD
import matplotlib.pyplot as plt
import numpy as np
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination import get_termination
from random import randint
from pymoo.termination.ftol import SingleObjectiveSpaceTermination
from pymoo.termination.robust import RobustTermination
from pymoo.termination.default import DefaultMultiObjectiveTermination

In [22]:
class tester():
    algorithm1_win_execution_time = 0
    algorithm1_win_convergence = 0
    algorithm2_win_execution_time = 0
    algorithm2_win_convergence = 0
    
    def __init__(self, algo1, algo2, problem):
        self.algo1 = algo1
        self.algo2 = algo2
        self.problem = problem
        self.res1_list = []
        self.res2_list = []
        self.res1_exec_time_list = []
        self.res2_exec_time_list = []
        self.res1_gd_list = []
        self.res2_gd_list = []

    def excute(self):
        pf = self.problem.pareto_front()
        ind = GD(pf)

        for i in range(1,11):
            res1 = minimize(self.problem,
                self.algo1,
                termination = DefaultMultiObjectiveTermination(xtol=1e-32,
                                                               cvtol=1e-32,
                                                               ftol=1e-6,
                                                               period=20,
                                                               n_max_gen=300000,
                                                               n_max_evals=100000
                                                            ),
                save_history=True,
                seed = i,
                verbose=False)
            
            res2 = minimize(self.problem,
                self.algo2,
                termination = DefaultMultiObjectiveTermination(xtol=1e-32,
                                                               cvtol=1e-32,
                                                               ftol=1e-6,
                                                               period=20,
                                                               n_max_gen=300000,
                                                               n_max_evals=100000
                                                            ),
                save_history=True,
                seed = i,
                verbose=False)
            
            self.res1_list.append(res1)
            self.res2_list.append(res2)
            self.res1_exec_time_list.append(res1.exec_time)
            self.res2_exec_time_list.append(res2.exec_time)

            res1_b = minimize(self.problem,
                self.algo1,
                termination = get_termination("n_eval", 50000),
                save_history=True,
                seed = i,
                verbose=False)
            
            res2_b = minimize(self.problem,
                self.algo2,
                termination = get_termination("n_eval", 50000),
                save_history=True,
                seed = i,

                verbose=False)
            if res1_b.F is None:
                self.res1_gd_list.append(None)
            else:
                self.res1_gd_list.append(ind(res1_b.F))
            if res2_b.F is None:
                self.res2_gd_list.append(None)
            else:
                self.res2_gd_list.append(ind(res2_b.F))
        
        #print("Best solution found by algorithm 1: \nX = %s\nOptimum found = %s\nCV = %s" % (res1.X, res1.F, res1.CV))
        #print("Best solution found by algorithm 2: \nX = %s\nOptimum found = %s\nCV = %s \n" % (res2.X, res2.F, res2.CV))
        
        algo1_avg_execute_time = sum(self.res1_exec_time_list) / len(self.res1_exec_time_list)
        algo2_avg_execute_time = sum(self.res2_exec_time_list) / len(self.res2_exec_time_list)
        algo1_execute_time_std = np.std(np.array(self.res1_exec_time_list))
        algo2_execute_time_std = np.std(np.array(self.res2_exec_time_list))
        
        
       
        
       
        
        print('Average execution time of algorithm 1:', algo1_avg_execute_time)
        print('Average execution time of algorithm 2:', algo2_avg_execute_time,'\n')

        print('Shortest execution time of algorithm 1:', min(self.res1_exec_time_list))
        print('Shortest execution time of algorithm 2:', min(self.res2_exec_time_list),'\n')
        
        print('Standard deviation of execution time of algorithm 1:', algo1_execute_time_std)
        print('Standard deviation of execution time of algorithm 2:', algo2_execute_time_std,'\n')
        
        if algo1_avg_execute_time < algo2_avg_execute_time:
            tester.algorithm1_win_execution_time += 1
            print("Algorithm 1 has shorter average execution time\n")
        else:
            tester.algorithm2_win_execution_time += 1
            print("Algorithm 2 has shorter average execution time\n")

        if None in self.res1_gd_list:
            print("No feasible solution found by algorithm 1")
        else:
            algo1_avg_gd = sum(self.res1_gd_list) / len(self.res1_gd_list)
            print("Avereage generational distance of algorithm 1:", algo1_avg_gd)
            print('Smallest generational distance of algorithm 1:', min(self.res1_gd_list))
            algo1_gd_std = np.std(np.array(self.res1_gd_list))
            print('Standard deviation of generational distance of algorithm 1:', algo1_gd_std,"\n")          
            if None in self.res2_list:
                tester.algorithm1_win_convergence += 1
        
        if None in self.res2_gd_list:
            print("No feasible solution found by algorithm 2")
        else:
            algo2_avg_gd = sum(self.res2_gd_list) / len(self.res2_gd_list)
            print("Avereage generational distance of algorithm 2:", algo2_avg_gd)
            print('Smallest generational distance of algorithm 2:', min(self.res2_gd_list))
            algo2_gd_std = np.std(np.array(self.res2_gd_list))
            print('Standard deviation of generational distance of algorithm 2:', algo2_gd_std,"\n")
            if None in self.res1_gd_list:
                tester.algorithm2_win_convergence += 1

        if not (None in self.res1_gd_list or None in self.res2_gd_list):
            if algo1_avg_gd < algo2_avg_gd:
                print("Algorithm 1 has a better approximation to the global optimum on average\n")
                tester.algorithm1_win_convergence += 1
            else:
                print("Algorithm 2 has a better approximation to the global optimum on average\n")
                tester.algorithm2_win_convergence += 1

    def visualize(self,res1,res2):
        opt1 = np.array([e.opt[0].F for e in res1.history])
        opt2 = np.array([e.opt[0].F for e in res2.history])
        pf = self.problem.pareto_front()
        n_eval1 = np.arange(len(res1.history))
        n_eval2 = np.arange(len(res2.history))
        plt.plot(n_eval1, opt1, label="Algorithm 1")
        plt.plot(n_eval2, opt2, label="Algorithm 2")
        plt.hlines(y=pf[0], xmin=0, xmax=max(n_eval1[-1],n_eval2[-1]), colors='purple', linestyles='--', lw=2)
        plt.title("Convergence Comparision")
        plt.xlabel("Genernation")
        plt.ylabel("Optimum in each generation")
        if pf[0] >= 0:
            plt.yscale("log")
        plt.legend()
        plt.show()
    
    def reset_count():
        tester.total_time_comparison = 0
        tester.total_convergence_comparison = 0
        tester.algorithm1_win_execution_time = 0
        tester.algorithm1_win_convergence = 0

In [3]:
algorithm1 = PSO()
algorithm2 = GA()
n_var = 20

In [4]:
ackley_tester = tester(algorithm1,algorithm2,get_problem("ackley", n_var=n_var, a=20, b=1/5, c=2 * np.pi))
ackley_tester.excute()
#ackley_tester.visualize()

Average execution time of algorithm 1: 3.5457268953323364
Average execution time of algorithm 2: 7.045603060722351 

Shortest execution time of algorithm 1: 0.8181107044219971
Shortest execution time of algorithm 2: 4.635232210159302 

Standard deviation of execution time of algorithm 1: 1.3887789859008324
Standard deviation of execution time of algorithm 2: 1.9468398271996963 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.002450681111394193
Smallest generational distance of algorithm 1: 3.8625547205128896e-11
Standard deviation of generational distance of algorithm 1: 0.007352042388805332 

Avereage generational distance of algorithm 2: 0.023485849929668224
Smallest generational distance of algorithm 2: 0.015702737194818628
Standard deviation of generational distance of algorithm 2: 0.005063124906111496 

Algorithm 1 has a better approximation to the global optimum on average



In [6]:
griewank_tester = tester(algorithm1,algorithm2,get_problem("griewank", n_var=n_var))
griewank_tester.excute()
#griewank_tester.visualize()

Average execution time of algorithm 1: 1.8597212314605713
Average execution time of algorithm 2: 6.989021301269531 

Shortest execution time of algorithm 1: 0.6398425102233887
Shortest execution time of algorithm 2: 5.135633707046509 

Standard deviation of execution time of algorithm 1: 1.486071378927032
Standard deviation of execution time of algorithm 2: 1.1412833298624059 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.23367893959177746
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 0.332257800339647 

Avereage generational distance of algorithm 2: 0.02778866028904551
Smallest generational distance of algorithm 2: 0.004815624556890441
Standard deviation of generational distance of algorithm 2: 0.015464593033517895 

Algorithm 2 has a better approximation to the global optimum on average



In [7]:
rastrigin_tester = tester(algorithm1,algorithm2,get_problem("rastrigin", n_var=n_var))
rastrigin_tester.excute()
#rastrigin_tester.visualize()

Average execution time of algorithm 1: 1.3278092622756958
Average execution time of algorithm 2: 7.542713356018067 

Shortest execution time of algorithm 1: 0.4227142333984375
Shortest execution time of algorithm 2: 5.109491586685181 

Standard deviation of execution time of algorithm 1: 1.438181156250368
Standard deviation of execution time of algorithm 2: 2.217688584285123 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 10.085512040407488
Smallest generational distance of algorithm 1: 0.9949592756015022
Standard deviation of generational distance of algorithm 1: 6.145119650498541 

Avereage generational distance of algorithm 2: 0.0028129929554467025
Smallest generational distance of algorithm 2: 0.0014046170907704436
Standard deviation of generational distance of algorithm 2: 0.0011953982288857212 

Algorithm 2 has a better approximation to the global optimum on average



In [21]:
rosenbrock_tester = tester(algorithm1,algorithm2,get_problem("rosenbrock", n_var=n_var))
rosenbrock_tester.excute()


n_gen  |  n_eval  |    f     |    S    |    w    |    c1    |    c2    |     f_avg     |     f_min    
     1 |       25 |        - |       - |  0.9000 |  2.00000 |  2.00000 |  9.242871E+03 |  3.761819E+03
     2 |       50 | -1.7E-01 |       3 |  0.2978 |  2.00000 |  2.01793 |  3.979530E+03 |  2.127930E+03
     3 |       75 |  0.06035 |       3 |  0.4382 |  1.99122 |  2.02945 |  1.873014E+03 |  8.649914E+02
     4 |      100 | -5.7E-02 |       3 |  0.3650 |  1.98138 |  2.04491 |  1.124434E+03 |  5.772204E+02
     5 |      125 | -1.9E-02 |       3 |  0.3879 |  1.96899 |  2.05511 |  9.340265E+02 |  5.012684E+02
     6 |      150 | -6.1E-02 |       3 |  0.3624 |  1.95819 |  2.07123 |  7.813006E+02 |  4.069415E+02
     7 |      175 |  0.00479 |       3 |  0.4030 |  1.94487 |  2.08005 |  6.275612E+02 |  2.905422E+02
     8 |      200 | -5.3E-02 |       3 |  0.3673 |  1.93365 |  2.08489 |  5.228597E+02 |  2.905422E+02
     9 |      225 | -3.9E-02 |       3 |  0.3758 |  1.92572 |  2.09208 | 

In [23]:
zakharov_tester = tester(algorithm1,algorithm2,get_problem("zakharov", n_var=n_var))
zakharov_tester.excute()

Average execution time of algorithm 1: 1.223110842704773
Average execution time of algorithm 2: 46.479407811164855 

Shortest execution time of algorithm 1: 0.1755673885345459
Shortest execution time of algorithm 2: 0.48760032653808594 

Standard deviation of execution time of algorithm 1: 0.7739163928749248
Standard deviation of execution time of algorithm 2: 68.3750107711978 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.004156040223900548
Smallest generational distance of algorithm 1: 6.125155375838453e-06
Standard deviation of generational distance of algorithm 1: 0.00893198662193912 

Avereage generational distance of algorithm 2: 1.934818369454462
Smallest generational distance of algorithm 2: 0.4675457433472518
Standard deviation of generational distance of algorithm 2: 1.046036747088875 

Algorithm 1 has a better approximation to the global optimum on average



In [24]:
g01_tester = tester(algorithm1,algorithm2,get_problem("g1"))
g01_tester.excute()

Average execution time of algorithm 1: 2.5084622859954835
Average execution time of algorithm 2: 25.401907420158388 

Shortest execution time of algorithm 1: 1.9576117992401123
Shortest execution time of algorithm 2: 2.1388566493988037 

Standard deviation of execution time of algorithm 1: 0.3601329801460491
Standard deviation of execution time of algorithm 2: 67.73257451365753 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 1.6328125
Smallest generational distance of algorithm 1: 0.0
Standard deviation of generational distance of algorithm 1: 1.1073099177313692 

Avereage generational distance of algorithm 2: 0.7185409892744946
Smallest generational distance of algorithm 2: 0.0006182285939004828
Standard deviation of generational distance of algorithm 2: 1.0955781466087653 

Algorithm 2 has a better approximation to the global optimum on average



In [25]:
g02_tester = tester(algorithm1,algorithm2,get_problem("g2"))
g02_tester.excute()

Average execution time of algorithm 1: 1.422611451148987
Average execution time of algorithm 2: 4.35578773021698 

Shortest execution time of algorithm 1: 0.8117940425872803
Shortest execution time of algorithm 2: 2.887737512588501 

Standard deviation of execution time of algorithm 1: 0.6056117518656393
Standard deviation of execution time of algorithm 2: 1.0039862336416812 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.3822517509428537
Smallest generational distance of algorithm 1: 0.2585376308028928
Standard deviation of generational distance of algorithm 1: 0.06860833261830072 

Avereage generational distance of algorithm 2: 0.046183436533672295
Smallest generational distance of algorithm 2: 0.01615944683827686
Standard deviation of generational distance of algorithm 2: 0.02544375290555106 

Algorithm 2 has a better approximation to the global optimum on average



In [26]:
g03_tester = tester(algorithm1,algorithm2,get_problem("g3"))
g03_tester.excute()

Average execution time of algorithm 1: 0.2600466012954712
Average execution time of algorithm 2: 29.843240475654603 

Shortest execution time of algorithm 1: 0.12035274505615234
Shortest execution time of algorithm 2: 0.41047096252441406 

Standard deviation of execution time of algorithm 1: 0.09712798581592165
Standard deviation of execution time of algorithm 2: 85.64875410429038 

Algorithm 1 has shorter average execution time

Avereage generational distance of algorithm 1: 0.6813953379960478
Smallest generational distance of algorithm 1: 0.3317180916601945
Standard deviation of generational distance of algorithm 1: 0.22339039263990637 

Avereage generational distance of algorithm 2: 0.8310796324287146
Smallest generational distance of algorithm 2: 0.4355922132760106
Standard deviation of generational distance of algorithm 2: 0.16197823385970975 

Algorithm 1 has a better approximation to the global optimum on average



In [ ]:
g04_tester = tester(algorithm1,algorithm2,get_problem("g4"))
g04_tester.excute()

In [ ]:
g05_tester = tester(algorithm1,algorithm2,get_problem("g5"))
g05_tester.excute()

In [ ]:
g06_tester = tester(algorithm1,algorithm2,get_problem("g6"))
g06_tester.excute()

In [ ]:
g07_tester = tester(algorithm1,algorithm2,get_problem("g7"))
g07_tester.excute()

In [ ]:
g08_tester = tester(algorithm1,algorithm2,get_problem("g8"))
g08_tester.excute()

In [ ]:
g09_tester = tester(algorithm1,algorithm2,get_problem("g9"))
g09_tester.excute()

In [ ]:
print(tester.algorithm1_win_execution_time + tester.algorithm2_win_execution_time)
print(tester.algorithm1_win_execution_time)
print(tester.algorithm2_win_execution_time)
print(tester.algorithm1_win_convergence + tester.algorithm2_win_convergence)
print(tester.algorithm1_win_convergence)
print(tester.algorithm2_win_convergence)
tester.reset_count()

1
1
0
1
1
0
